*词向量*

In [1]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('model_191115_1')

*词频*

In [2]:
total = 0
for k in model.wv.vocab.keys():
    total += model.wv.vocab[k].count
def get_fre(w):
    return model.wv.vocab[w].count / total

*SIF*

In [3]:
import numpy as np
from sklearn.decomposition import PCA

In [4]:
# Step 1
def sif_s1(s:list, a=1e-3) -> np.array:
    v = np.zeros(model.wv['算法'].shape)
    count = 0
    for w in s:
        if w not in model.wv: continue
        we_w = a / (a + get_fre(w)) * model.wv[w]
        v += we_w
        count += 1
    if count > 0:
        return v / count
    else:
        return v

# Step 2
def sif_s2(s:np.array) -> np.array:
    pca = PCA(n_components=1)
    pca.fit(s)
    pc = pca.components_
    return s - s.dot(pc.T) * pc

*余弦相似度*

In [5]:
def relative(sentences, title, article, weight=0.5):
    def cosine_v(v1, v2):
        num = np.dot(v1, v2.T)
        demon = np.linalg.norm(v1) * np.linalg.norm(v2)
        return num / demon
    
    c = []
    for i in range(sentences.shape[0]):
        s = sentences[i, :]
        c_i = weight * cosine_v(s, article.T) + (1 - weight) * cosine_v(s, title.T)
        c.append(c_i)
    return c

*输入数据的读取和切分*

>模型上线后会有读取数据的接口，这里本地测试时直接从新闻库随机取一篇

In [6]:
import pandas as pd
import random

In [26]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [18]:
news.shape

(89611, 7)

In [33]:
longNews = news[news['content'].str.len()>1000]

In [15]:
def get_random_news(new, seed=5):
    random.seed(seed)
    i = random.randint(0, news.shape[0] - 1)
    return news.iloc[i,:]['title'], news.iloc[i,:]['content']

In [10]:
title, content = get_random_news(news, 20)

In [11]:
title

'（法治）“结石宝宝”父亲“敲诈勒索”案再审改判无罪'

In [12]:
content

'\u3000\u3000新华社广州４月７日电（记者毛一竹）２００８年三聚氰胺奶粉事件中受害女童的父亲郭利，因与奶粉企业交涉赔偿问题被法院认定构成敲诈勒索罪，判处有期徒刑５年。７日，广东省高级人民法院对该案进行再审宣判，改判郭利无罪，可依法申请国家赔偿。\\n\u3000\u3000２００８年９月，“施恩”牌奶粉被曝出部分批次产品含有三聚氰胺。郭利因女儿曾食用过该品牌奶粉，带女儿到医院检查，结果显示：“双肾中央集合系统内可见数个点状强回声”。\\n\u3000\u3000之后，郭利将家中剩下的和新购买的部分“施恩”牌奶粉送检，检出两个批次奶粉三聚氰胺含量较高。随后，郭利多次找销售商和施恩公司索赔，并向媒体曝光。\\n\u3000\u3000２００９年６月１３日，施恩公司与郭利达成和解协议，施恩公司补偿郭利一方人民币４０万元，郭利出具书面材料表示不再追诉并放弃赔偿要求。\\n\u3000\u3000２００９年６月２５日，北京电视台播出了题为《一个男人，如何使“施恩”奶粉低头》的报道，内容主要是郭利向该台反映“施恩”奶粉问题。同年６月２９日，施恩公司及其控股股东广东雅士利公司派员主动与郭利取得联系。在双方沟通的过程中，郭利提出要求对方再赔偿３００万元。雅士利公司认为郭利提出过高要求是对其敲诈勒索遂报案，郭利被抓。\\n\u3000\u3000２０１０年１月，潮安县法院一审以敲诈勒索罪，判处郭利有期徒刑５年。次月，潮州市中院作出二审裁定，驳回郭利的上诉，维持原判。当年５月，广东高院作出再审决定，指令潮州中院再审。潮州中院于当年１２月再审后，又一次裁定维持原判。此后，郭利的父母提出申诉，广东高院审查后按照审判监督程序提审该案。\\n\u3000\u3000广东高院再审认为：从本案发生、发展的过程看，尚不能认定郭利的行为性质超出民事纠纷的范畴。现有证据不足以证明郭利具有非法占有他人财物的目的，也不足以证明郭利实施了敲诈勒索行为。故判决撤销潮州中院及潮安县法院原裁判，改判原审被告人郭利无罪。审判长当庭告知郭利可依法申请国家赔偿。\\n\u3000\u3000“等了９年，此时唯一想说的就是真理常在、法理常在。”宣判后，郭利告诉记者，他将继续通过法律渠道申请国家赔偿，维护自身权利。\\n\u3000\u3000中山大学法学院教授聂立泽说：“通过本案的纠错，我们看到了实现个案公正的目

>**句子切分规则**
1. 默认新闻是中文，且标点符号都正确运用
2. 以`。；！？`为切分点(如果后面紧跟着下引用号`”`，则包含该符号在内)，在切分点后面插入特殊符号(这里使用`@%#`3个特殊符号一起作为分隔符，正常情况下新闻不可能出现3个符号一起的情况)
3. 切分后去掉首尾的空格，然后测试是否是一个句子（将句子里所有符号换为空格后查看是否存在字符），并去掉字符少于5的句子，重组所有句子

In [7]:
import re

In [8]:
def cut_content(content, cut_point='。；！？', spliter='@%#'):
    # insert spliters
    new_content = ''
    i = 0
    while i < len(content):
        new_content += content[i]
        if content[i] in cut_point:
            if content[i+1] == '”':
                i += 1
                new_content += content[i]
            new_content += spliter
        i += 1
    
    # cut
    sentences = re.split('[{}]'.format(spliter), new_content)
    
    # filter
    new_sentences = []
    for s in sentences:
        s = re.sub('\\\\n|[\n\u3000\r]', '', s)
        s = s.strip()
        if len(s) > 5: 
            new_sentences.append(s)
        
    return new_sentences

In [16]:
cut_content(content)

['新华社广州４月７日电（记者毛一竹）２００８年三聚氰胺奶粉事件中受害女童的父亲郭利，因与奶粉企业交涉赔偿问题被法院认定构成敲诈勒索罪，判处有期徒刑５年。',
 '７日，广东省高级人民法院对该案进行再审宣判，改判郭利无罪，可依法申请国家赔偿。',
 '２００８年９月，“施恩”牌奶粉被曝出部分批次产品含有三聚氰胺。',
 '郭利因女儿曾食用过该品牌奶粉，带女儿到医院检查，结果显示：“双肾中央集合系统内可见数个点状强回声”。',
 '之后，郭利将家中剩下的和新购买的部分“施恩”牌奶粉送检，检出两个批次奶粉三聚氰胺含量较高。',
 '随后，郭利多次找销售商和施恩公司索赔，并向媒体曝光。',
 '２００９年６月１３日，施恩公司与郭利达成和解协议，施恩公司补偿郭利一方人民币４０万元，郭利出具书面材料表示不再追诉并放弃赔偿要求。',
 '２００９年６月２５日，北京电视台播出了题为《一个男人，如何使“施恩”奶粉低头》的报道，内容主要是郭利向该台反映“施恩”奶粉问题。',
 '同年６月２９日，施恩公司及其控股股东广东雅士利公司派员主动与郭利取得联系。',
 '在双方沟通的过程中，郭利提出要求对方再赔偿３００万元。',
 '雅士利公司认为郭利提出过高要求是对其敲诈勒索遂报案，郭利被抓。',
 '２０１０年１月，潮安县法院一审以敲诈勒索罪，判处郭利有期徒刑５年。',
 '次月，潮州市中院作出二审裁定，驳回郭利的上诉，维持原判。',
 '当年５月，广东高院作出再审决定，指令潮州中院再审。',
 '潮州中院于当年１２月再审后，又一次裁定维持原判。',
 '此后，郭利的父母提出申诉，广东高院审查后按照审判监督程序提审该案。',
 '广东高院再审认为：从本案发生、发展的过程看，尚不能认定郭利的行为性质超出民事纠纷的范畴。',
 '现有证据不足以证明郭利具有非法占有他人财物的目的，也不足以证明郭利实施了敲诈勒索行为。',
 '故判决撤销潮州中院及潮安县法院原裁判，改判原审被告人郭利无罪。',
 '审判长当庭告知郭利可依法申请国家赔偿。',
 '“等了９年，此时唯一想说的就是真理常在、法理常在。”',
 '宣判后，郭利告诉记者，他将继续通过法律渠道申请国家赔偿，维护自身权利。',
 '中山大学法学院教授聂立泽说：“通过本案的纠错，我们看到了实现个案公正的目标正在落实，看到了人权刑事司法保障的逐步完

In [37]:
sentences = cut_content(content)

In [38]:
sentences[0]

'新华社广州４月７日电（记者毛一竹）２００８年三聚氰胺奶粉事件中受害女童的父亲郭利，因与奶粉企业交涉赔偿问题被法院认定构成敲诈勒索罪，判处有期徒刑５年。'

*句子相关性计算*

In [9]:
import jieba

In [12]:
def get_c(title, sentences, sif_a=1e-3, weight=0.5):
    punctuation = '！？｡。，＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘\'‛“”„‟…‧﹏' + '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~·ʔ•'
    article = []
    
    def sentence_to_words(s):
        s = re.sub('[{}]'.format(punctuation+'\n'), ' ', s)
        words = jieba.cut(s)
        return [w for w in words if w.strip()]
    
    title = sentence_to_words(title)
    for i in range(len(sentences)):
        words = sentence_to_words(sentences[i])
        article += words
        sentences[i] = [sentences[i], words]
    title_v = sif_s1(title, a=sif_a)
    article_v = sif_s1(article, a=sif_a)
    sentences_v = np.zeros([len(sentences), 250])
    for i in range(len(sentences)):
        sentences_v[i,:] = sif_s1(sentences[i][1])
    sentences_v = sif_s2(sentences_v)
    
    c = relative(sentences_v, title_v, article_v, weight)
    # knn
    knn_c = []
    def left_(i):
        if i <= 2:
            return 0
        else:
            return i - 2
    for i in range(len(c)):
        c_i = sum(c[left_(i): i+3]) / len(c[left_(i): i+3])
        knn_c.append(c_i)
    
    for i in range(len(knn_c)):
        sentences[i] = [sentences[i][0], knn_c[i], i]
    
    
    return sentences

In [39]:
s_c = get_c(title, sentences)

In [40]:
s_c

[['新华社广州４月７日电（记者毛一竹）２００８年三聚氰胺奶粉事件中受害女童的父亲郭利，因与奶粉企业交涉赔偿问题被法院认定构成敲诈勒索罪，判处有期徒刑５年。',
  0.6162444676998905,
  0],
 ['７日，广东省高级人民法院对该案进行再审宣判，改判郭利无罪，可依法申请国家赔偿。', 0.5809857634172676, 1],
 ['２００８年９月，“施恩”牌奶粉被曝出部分批次产品含有三聚氰胺。', 0.5610702546501919, 2],
 ['郭利因女儿曾食用过该品牌奶粉，带女儿到医院检查，结果显示：“双肾中央集合系统内可见数个点状强回声”。', 0.5384087953997618, 3],
 ['之后，郭利将家中剩下的和新购买的部分“施恩”牌奶粉送检，检出两个批次奶粉三聚氰胺含量较高。', 0.5099531468119577, 4],
 ['随后，郭利多次找销售商和施恩公司索赔，并向媒体曝光。', 0.5289939317926361, 5],
 ['２００９年６月１３日，施恩公司与郭利达成和解协议，施恩公司补偿郭利一方人民币４０万元，郭利出具书面材料表示不再追诉并放弃赔偿要求。',
  0.5139432906986376,
  6],
 ['２００９年６月２５日，北京电视台播出了题为《一个男人，如何使“施恩”奶粉低头》的报道，内容主要是郭利向该台反映“施恩”奶粉问题。',
  0.5298126727687059,
  7],
 ['同年６月２９日，施恩公司及其控股股东广东雅士利公司派员主动与郭利取得联系。', 0.5450547190650127, 8],
 ['在双方沟通的过程中，郭利提出要求对方再赔偿３００万元。', 0.5288834275696386, 9],
 ['雅士利公司认为郭利提出过高要求是对其敲诈勒索遂报案，郭利被抓。', 0.507825213703314, 10],
 ['２０１０年１月，潮安县法院一审以敲诈勒索罪，判处郭利有期徒刑５年。', 0.524493974521623, 11],
 ['次月，潮州市中院作出二审裁定，驳回郭利的上诉，维持原判。', 0.5148533144680243, 12],
 ['当年５月，广东高院作出再审决定，指令潮州中院再审。', 0.5145692212475

*返回排序前5的句子*

In [24]:
topn = sorted([n[0] for n in sorted(s_c, key=lambda x: x[1], reverse=True)[:5]])
for i in range(len(topn)):
    print(s_c[i][0])

新华社广州４月７日电（记者毛一竹）２００８年三聚氰胺奶粉事件中受害女童的父亲郭利，因与奶粉企业交涉赔偿问题被法院认定构成敲诈勒索罪，判处有期徒刑５年。
７日，广东省高级人民法院对该案进行再审宣判，改判郭利无罪，可依法申请国家赔偿。
２００８年９月，“施恩”牌奶粉被曝出部分批次产品含有三聚氰胺。
郭利因女儿曾食用过该品牌奶粉，带女儿到医院检查，结果显示：“双肾中央集合系统内可见数个点状强回声”。
之后，郭利将家中剩下的和新购买的部分“施恩”牌奶粉送检，检出两个批次奶粉三聚氰胺含量较高。


### 整合模型

>可调参数有:
>>SIF模型的a, 一般为1e-3或1e-5<br>
标题和内容的余弦相似度比例\[0-1\]<br>
KNN平滑时取值范围和加权比例<br>

>固定参数有:
>>词向量维度：250<br>
所有可能出现的标点符号punctuation<br>

In [47]:
# 可调参数
sif_a = 1e-3
weight = 0.5

In [13]:
def article_summary(title, content, n=5, sif_a=1e-3, weight=0.5):
    sentences = cut_content(content)
    sentences_c = get_c(title, sentences, sif_a, weight)
    topn = [s[0] for s in sorted([s for s in sorted(sentences_c, key=lambda x: x[1], reverse=True)][:n], key=lambda x:x[2])]
    return ''.join(topn)

In [49]:
article_summary(title, content)

'新华社广州４月７日电（记者毛一竹）２００８年三聚氰胺奶粉事件中受害女童的父亲郭利，因与奶粉企业交涉赔偿问题被法院认定构成敲诈勒索罪，判处有期徒刑５年。７日，广东省高级人民法院对该案进行再审宣判，改判郭利无罪，可依法申请国家赔偿。２００８年９月，“施恩”牌奶粉被曝出部分批次产品含有三聚氰胺。广东高院再审认为：从本案发生、发展的过程看，尚不能认定郭利的行为性质超出民事纠纷的范畴。现有证据不足以证明郭利具有非法占有他人财物的目的，也不足以证明郭利实施了敲诈勒索行为。'

In [120]:
title, content = get_random_news(news, 0)

In [36]:
title, content = get_random_news(longNews, 100)
print(title)
print(content)

今日“君”再来——邓丽君借助虚拟技术在舞台上“复活”
新华社台北4月7日新媒体专电（记者马玉洁;章利新）台湾著名歌星邓丽君在逝世20多年后，将借助顶尖虚拟技术“复活”，并主演一部奇幻舞台剧，在五月的台北与她的众多粉丝“重逢”。\n邓丽君的歌曲曾是华人世界几代人的记忆，她的突然身故也一直让人唏嘘不已。近年来，随着数字虚拟技术的深入运用，依靠技术力量，邓丽君多次“登上”舞台，还举办了“如果能许一个愿”逝世20周年演唱会。\n　　这是在台北市三创生活园区举办的”虚拟人邓丽君音乐奇幻SHOW“记者会现场（4月6日摄）\n在即将于5月上演的舞台剧《今日君再来：虚拟人邓丽君音乐奇幻SHOW》中，由虚拟人像技术打造的邓丽君将与真人演员共同演出。这部剧以邓丽君喜爱的蝴蝶为意象，讲述了一代歌后美丽重生的故事。\n剧中邓丽君的形象由好莱坞视觉特效公司数字王国创作而成，透过整合面部捕捉、动作捕捉及特效技术，并辅以舞台设计与同台演员的精湛演出，将一代巨星的风采以全新舞台剧形式完整重现于观众身边。\n此外，为了创造全新的5D多重感官体验，这部舞台剧也将打破传统，配合故事发展，把观众融入表演当中，并会与演员有近距离互动，享受全新的娱乐体验。\n邓丽君文教基金会董事长、邓丽君的哥哥邓长富表示，邓丽君曾给予我们的众多难忘回忆、音乐与感动，将借助科技的力量不断延续，并焕发出新的喜乐与生机。“对我们家人以至全球歌迷而言，这都令人非常欣慰和期待。”\n　　这是在台北市三创生活园区举办的”虚拟人邓丽君音乐奇幻SHOW“记者会现场（4月6日摄）\n2013年，数字王国在周杰伦演唱会中通过虚拟人像技术让邓丽君“复活”，并与周杰伦同台合唱了三首歌曲，引起一时轰动。\n当年的这段表演虽然只有短短的3分半钟，却是由数字王国45名特效师经过2个多月的时间，耗资超过1亿元新台币（约合2280万人民币）打造而成。\n数字王国执行董事及行政总裁谢安说，如何尽可能真实地还原邓丽君讲话的神韵，抓住她的优雅，是此次舞台剧最大的难题。“每个人的牙齿、讲话的方式、皮肤的皱褶都不一样，所以很难完全复制她的神态。此次我们邀请了大约200多名艺术家专门负责揣摩邓小姐的神韵。”\n《今日君再来：虚拟人邓丽君音乐奇幻SHOW》将于5月6日至10日在台北三创生活园区上演。\n


In [41]:
article_summary(title, content, n=5, sif_a=1e-3, weight=0.5)

'邓丽君的歌曲曾是华人世界几代人的记忆，她的突然身故也一直让人唏嘘不已。这是在台北市三创生活园区举办的”虚拟人邓丽君音乐奇幻SHOW“记者会现场（4月6日摄）在即将于5月上演的舞台剧《今日君再来：虚拟人邓丽君音乐奇幻SHOW》中，由虚拟人像技术打造的邓丽君将与真人演员共同演出。这部剧以邓丽君喜爱的蝴蝶为意象，讲述了一代歌后美丽重生的故事。剧中邓丽君的形象由好莱坞视觉特效公司数字王国创作而成，透过整合面部捕捉、动作捕捉及特效技术，并辅以舞台设计与同台演员的精湛演出，将一代巨星的风采以全新舞台剧形式完整重现于观众身边。邓丽君文教基金会董事长、邓丽君的哥哥邓长富表示，邓丽君曾给予我们的众多难忘回忆、音乐与感动，将借助科技的力量不断延续，并焕发出新的喜乐与生机。'

In [160]:
article_summary(title, content, n=9, sif_a=1e-3, weight=0.4)

'新华社杭州５月１７日电题：智能经济，下一步往哪走？新华社记者朱涵、黄筱、裘立华随着人工智能和物联网、大数据、云计算的协同发展，智能经济正在成为经济创新发展和转型升级的新引擎。智能经济将带来什么变化，又将走向何方？正在浙江余姚市召开的第四届中国机器人峰会暨智能经济人才峰会上，全球人工智能专家和业内人士共话智能经济。智能制造迈进数字工厂仅凭两只机械手，自如切换十多套复杂夹具，就将大大小小的精细零部件组装进一件产品……第四届中国机器人峰会暨智能经济人才峰会上展出的一条数字工厂流水线前人头攒动。比如在工厂广泛使用的机械臂将能够在办公室、银行等场所处理烦琐、重复性的工作。”凯文·凯利认为，人工智能将变得越来越具有“逻辑”。凯文·凯利说。浙江大学人工智能研究所副教授吴飞说。'